In [1]:
import pandas as pd
import numpy as np

第八章 数据规整：连接、联合与重塑
在很多应用中，数据可能分布在多个文件或数据库中，抑或以某种不易于分析的格式进行排列。本章关注于数据的联合、连接及重排。

8.1 分层索引
分层索引时pandas的重要特性，允许你在一个轴向上拥有多个（两个或两个以上）索引层级。笼统的说，分层索引提供了一种在更低维度的形式中处理更高维度数据的方式。

In [5]:
data  = pd.Series(np.random.randn(9), index = [['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'], 
                                              [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.635528
   2   -0.121646
   3   -0.600248
b  1    1.165142
   3   -0.601310
c  1   -1.280104
   2    0.944530
d  2   -1.437216
   3   -0.431435
dtype: float64

In [6]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

In [8]:
# 部分索引，允许简洁地选择出数据的子集
print(data['b'])
print(data['b':'c'])
print(data.loc[['b', 'd']])

# 在内部层级中进行选择
print(data.loc[:, 2])

1    1.165142
3   -0.601310
dtype: float64
b  1    1.165142
   3   -0.601310
c  1   -1.280104
   2    0.944530
dtype: float64
b  1    1.165142
   3   -0.601310
d  2   -1.437216
   3   -0.431435
dtype: float64
a   -0.121646
c    0.944530
d   -1.437216
dtype: float64


In [10]:
# 分层索引在重塑数据和数据透视表等分组操作中扮演了重要角色。例如，可以使用unstack方法将数据在 DataFrame中重新排列
print(data.unstack())

# unstack的反操作是stack()
print(data.unstack().stack())

          1         2         3
a -0.635528 -0.121646 -0.600248
b  1.165142       NaN -0.601310
c -1.280104  0.944530       NaN
d       NaN -1.437216 -0.431435
a  1   -0.635528
   2   -0.121646
   3   -0.600248
b  1    1.165142
   3   -0.601310
c  1   -1.280104
   2    0.944530
d  2   -1.437216
   3   -0.431435
dtype: float64


In [15]:
# DataFrame中，每个轴都可以用于分层索引
frame = pd.DataFrame(np.arange(12).reshape((4,3)), index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]], 
                                         columns=[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [16]:
# 分层的层级可以有名称（可以是字符串或Python对象）
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [17]:
# 部分索引，选出列中的组
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [20]:
# 一个MultiIndex 对象可以使用其自身的构造函数创建并复用。
multi_index = pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]], names = ['key1', 'key2'])
multi_cols = pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']], names = ['state', 'color'])
frame = pd.DataFrame(np.arange(12).reshape((4,3)), index=multi_index, columns=multi_cols)
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

8.1.1 重排序和层级排序

In [21]:
# swaleveel 接收两个层级序号或层级名称，返回一个进行了层级变更的新对象
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [23]:
# sort_index 只能在单一层级上对数据进行排序。在进行层级变变换时，使用sort_index以使得结果按照层级进行字典排序
print(frame.sort_index(level=1))
print(frame.swaplevel(0, 1).sort_index(level=0))

# 如果索引按照字典顺序从最外层开始排序，那么数据选择性能会更好--调用sort_index(level=0)或sort_index可以得到这样的结果

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11
state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11


8.1.2 按层级进行汇总统计

In [24]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [27]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

8.1.3 使用DataFrame的列进行索引
通常我们不会使用DataFrame中一个或多个列作为索引；反而你可能想要将索引移动到DataFrame的列中。

In [28]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [31]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [33]:
# 默认情况下，这些列会从DataFrame中移除，也可以将他们保留在DataFrame中
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [34]:
# reset_index是set_index的反操作，分层索引的索引层级会被移动到列中
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


8.2 联合与合并数据集
pandas.merge:根据一个或多个键进行连接
pandas.concat:使对象在轴向上进行黏合或“堆叠”
combine_first:实例方法允许将重叠的数据拼接在一起

8.2.1 数据库风格的DataFrame 连接
合并或连接操作通过一个或多个键连接行联合数据集。这些操作是关系型数据库的核心内容（例如SQL数据库）。

In [35]:
df1 = pd.DataFrame({'key':['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})
df2 = pd.DataFrame({'key':['a', 'b', 'd'], 'data2': range(3)})
print(df1)
print(df2)

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6
  key  data2
0   a      0
1   b      1
2   d      2


In [36]:
# 多对一连接
pd.merge(df1, df2)

# 没有指定在那一列上进行连接，如果连接的键信息没有指定，merge会自动将重叠列名作为连接的键

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [37]:
# 但是，显式地指定连接键才是好的实现
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [38]:
# 如果每个对象的列名是不同的，可以分别为他们指定列名
df3 = pd.DataFrame({'lkey':['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})
df4 = pd.DataFrame({'rkey':['a', 'b', 'd'], 'data2': range(3)})

pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [39]:
# 默认情况，merge做的是内连接('inner join'),结果中的键是两张表的交集
# how = 'outer' 对两张表都有的键的并集进行联合
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [42]:
df1 = pd.DataFrame({'key':['b', 'b', 'a', 'c', 'a', 'b'], 'data1': range(6)})
df2 = pd.DataFrame({'key':['a', 'b', 'a', 'b', 'd'], 'data2': range(5)})
print(df1)
print(df2)

# how='left' 对所有左表的键进行联合
print(pd.merge(df1, df2, on='key', how='left'))
# how='right' 对所有左表的键进行联合
print(pd.merge(df1, df2, on='key', how='right'))

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   b      5
  key  data2
0   a      0
1   b      1
2   a      2
3   b      3
4   d      4
   key  data1  data2
0    b      0    1.0
1    b      0    3.0
2    b      1    1.0
3    b      1    3.0
4    a      2    0.0
5    a      2    2.0
6    c      3    NaN
7    a      4    0.0
8    a      4    2.0
9    b      5    1.0
10   b      5    3.0
   key  data1  data2
0    b    0.0      1
1    b    1.0      1
2    b    5.0      1
3    b    0.0      3
4    b    1.0      3
5    b    5.0      3
6    a    2.0      0
7    a    4.0      0
8    a    2.0      2
9    a    4.0      2
10   d    NaN      4


In [43]:
# 多对多连接是行的笛卡尔内积
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [47]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})
print(left)
print(right)
pd.merge(left, right, on=['key1', 'key2'], how='outer')

  key1 key2  lval
0  foo  one     1
1  foo  two     2
2  bar  one     3
  key1 key2  rval
0  foo  one     4
1  foo  one     5
2  bar  one     6
3  bar  two     7


,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [48]:
# 合并操作中最后一个要考虑的问题是如何处理重叠列名
print(pd.merge(left, right, on="key1"))
print(pd.merge(left, right, on="key1", suffixes=('_left', '_right')))

  key1 key2_x  lval key2_y  rval
0  foo    one     1    one     4
1  foo    one     1    one     5
2  foo    two     2    one     4
3  foo    two     2    one     5
4  bar    one     3    one     6
5  bar    one     3    two     7
  key1 key2_left  lval key2_right  rval
0  foo       one     1        one     4
1  foo       one     1        one     5
2  foo       two     2        one     4
3  foo       two     2        one     5
4  bar       one     3        one     6
5  bar       one     3        two     7


In [51]:
# left_index right_index 行索引作为连接键
pd.merge(left, right,left_index=True, right_index=True)

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,two,2,one,5
2,bar,one,3,one,6


In [52]:
# sort参数通过连接键按字母顺序对合并的数据进行排序。在默认情况下为true（在大数据集上某些情况下禁用该功能可以获得更好的性能）
print(pd.merge(left, right, on="key1", suffixes=('_left', '_right')))
print(pd.merge(left, right, on="key1", suffixes=('_left', '_right'), sort=False))

  key1 key2_left  lval key2_right  rval
0  foo       one     1        one     4
1  foo       one     1        one     5
2  foo       two     2        one     4
3  foo       two     2        one     5
4  bar       one     3        one     6
5  bar       one     3        two     7
  key1 key2_left  lval key2_right  rval
0  foo       one     1        one     4
1  foo       one     1        one     5
2  foo       two     2        one     4
3  foo       two     2        one     5
4  bar       one     3        one     6
5  bar       one     3        two     7


In [54]:
# indicator 添加一个特鳄属的列_merge，指示每一行的来源；值将根据每行中连接数据的来源分别为“left_only”、"right_only"或"both"
pd.merge(left, right, on="key1", suffixes=('_left', '_right'), indicator=True)

,key1,key2_left,lval,key2_right,rval,_merge
0,foo,one,1,one,4,both
1,foo,one,1,one,5,both
2,foo,two,2,one,4,both
3,foo,two,2,one,5,both
4,bar,one,3,one,6,both
5,bar,one,3,two,7,both


8.2.2 根据索引合并
在某些情况下，DataFrame 中用于合并的键是它的索引。

In [55]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
print(left1)
print(right1)

  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5
   group_val
a        3.5
b        7.0


In [56]:
pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [57]:
# 默认的合并方法是按键相交，可以使用外连接来进行合并
pd.merge(left1, right1, left_on="key", right_index=True, how="outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [58]:
# 在多层索引数据情况下，事情会更复杂，在索引上连接是一个隐式的多建合作
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                              'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])
print(lefth)
print(righth)

     key1  key2  data
0    Ohio  2000   0.0
1    Ohio  2001   1.0
2    Ohio  2002   2.0
3  Nevada  2001   3.0
4  Nevada  2002   4.0
             event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11


In [59]:
# 在这种情况下，你必须以列表的方式指明合并所需多个列（请注意使用how="outer"处理重复的索引值）
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [60]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how="outer")

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [61]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])
print(left2)
print(right2)

   Ohio  Nevada
a   1.0     2.0
c   3.0     4.0
e   5.0     6.0
   Missouri  Alabama
b       7.0      8.0
c       9.0     10.0
d      11.0     12.0
e      13.0     14.0


In [62]:
# 使两边的索引进行合并
pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [63]:
# DataFrame Joib实例方法，用于按照索引合并。该方法可以用于合并多个索引相同或相似但没有重叠列对象
left2.join(right2, how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [64]:
# 由于历史原因，DataFrame的join方法进行连接键上的左连接，完全保留左边的DataFrame的行索引。
# 它还支持在调用DataFrame的某一列上连接传递的DataFrame的索引
left1.join(right1, on="key")

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [ ]:
# 简单索引-索引合并, join 无重叠列的DataFrame对象
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])
print(left2)
print(right2)
print(another)
print(left2.join([right2, another]))
print(left2.join([right2, another], how="outer", sort=False))

 8.2.3 沿轴向连接
另一种数据组合操作可互换地凭借、绑定或堆叠。Numpy的concatenate函数可以在Numpy数组上实现该功能

In [6]:
arr = np.arange(12).reshape((3,4))
print(arr)
print(np.concatenate([arr, arr], axis=1))
print(np.concatenate([arr, arr, arr], axis=0))

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
[[ 0  1  2  3  0  1  2  3]
 [ 4  5  6  7  4  5  6  7]
 [ 8  9 10 11  8  9 10 11]]
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


In [9]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

# 默认concat方法沿axis=0的轴向生效
print(pd.concat([s1, s2, s3], axis=1))
print(pd.concat([s1, s2, s3], axis=1, join='inner'))

     0    1    2
a  0.0  NaN  NaN
b  1.0  NaN  NaN
c  NaN  2.0  NaN
d  NaN  3.0  NaN
e  NaN  4.0  NaN
f  NaN  NaN  5.0
g  NaN  NaN  6.0
Empty DataFrame
Columns: [0, 1, 2]
Index: []


d:\programfiles\python\python37\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [18]:
s4 = pd.concat([s1, s3])
print(s4)

print(pd.concat([s1, s4], axis=1, sort=False))
print(pd.concat([s1, s4], axis=1, sort=False, join='inner'))

# 可以使用join_axes来指定用于连接其他轴向的轴
print(pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']], sort=False))

# 在连接轴向上创建多层索引，可以使用keys参数来实现
result = pd.concat([s1, s2, s3], keys=['one', 'two', 'three'], sort=False)
print(result)
print(result.unstack())

a    0
b    1
f    5
g    6
dtype: int64
     0  1
a  0.0  0
b  1.0  1
f  NaN  5
g  NaN  6
   0  1
a  0  0
b  1  1
     0    1
a  0.0  0.0
c  NaN  NaN
b  1.0  1.0
e  NaN  NaN
one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64
         a    b    c    d    e    f    g
one    0.0  1.0  NaN  NaN  NaN  NaN  NaN
two    NaN  NaN  2.0  3.0  4.0  NaN  NaN
three  NaN  NaN  NaN  NaN  NaN  5.0  6.0


In [21]:
# 沿轴向axis=1连接series的时候，keys则成为DataFrame的列头
print(pd.concat([s1,s2,s3], axis=1, keys=['one', 'two', 'three'], sort=False))

   one  two  three
a  0.0  NaN    NaN
b  1.0  NaN    NaN
c  NaN  2.0    NaN
d  NaN  3.0    NaN
e  NaN  4.0    NaN
f  NaN  NaN    5.0
g  NaN  NaN    6.0


In [26]:
# 相同的操作可以利用到DataFrame
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])
print(df1)
print(df2)

print(pd.concat([df1, df2], axis=1, keys=['level1', 'level2'], sort= False))
print(pd.concat({'level1':df1, 'leveel2':df2}, axis=1, sort=False))

   one  two
a    0    1
b    2    3
c    4    5
   three  four
a      5     6
c      7     8
  level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0
  leveel2      level1    
    three four    one two
a     5.0  6.0      0   1
c     7.0  8.0      4   5
b     NaN  NaN      2   3


In [27]:
# 一些额外的负责多层索引生成表，如names参数命名生成的轴层级
pd.concat([df1, df2], axis=1, keys=['level1', 'leevel2'], names=['upper', 'lower'], sort=False)

upper level1     leevel2     
lower    one two   three four
a          0   1     5.0  6.0
b          2   3     NaN  NaN
c          4   5     7.0  8.0

In [28]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
print(df1)
print(df2)

          a         b         c         d
0  0.904761  0.693932 -0.045367 -1.495760
1  0.726384 -1.771464  0.327459  2.176863
2 -0.425993 -0.053891  0.483667 -1.375233
         b         d         a
0 -0.82560 -0.288751  1.168114
1 -1.98475 -0.882758  0.516285


In [31]:
# ignore_index 不沿着连接轴保留索引，产生一段新的索引
pd.concat([df1, df2], ignore_index=True, sort=False)

,a,b,c,d
0,0.904761,0.693932,-0.045367,-1.495760
1,0.726384,-1.771464,0.327459,2.176863
2,-0.425993,-0.053891,0.483667,-1.375233
3,1.168114,-0.825600,NaN,-0.288751
4,0.516285,-1.984750,NaN,-0.882758


8.2.4 联合重叠数据
还有另一个数据联合场景，既不是合并操作，也不是连接操作。可能有两个数据集，这两个数据集的所有全部或部分重叠。

In [36]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
print(a)
print(b)
print(np.where(pd.isnull(a), b, a))

# Series有一个combine_first方法，该方法可以等价于下面这种使用pandas常见数据对其逻辑的轴向操作
print(a.combine_first(b))

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64
f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64
[0.  2.5 2.  3.5 4.5 5. ]
f    0.0
e    2.5
d    2.0
c    3.5
b    4.5
a    5.0
dtype: float64


In [38]:
# 在DataFrame中，combine_first逐列做相同的操作，因此你可以认为它是根据你传入的对象来修补调用对象的缺失值
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})
print(df1)
print(df2)
df1.combine_first(df2)

     a    b   c
0  1.0  NaN   2
1  NaN  2.0   6
2  5.0  NaN  10
3  NaN  6.0  14
     a    b
0  5.0  NaN
1  4.0  3.0
2  NaN  4.0
3  3.0  6.0
4  7.0  8.0


,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


8.3 重塑和透视
重排列表格型数据有多种基础操作。这些操作被称为重塑或透视

8.3.1 使用多层索引进行重塑
多层索引在DataFrame中提供了一种一致性方式用于重排数据。以下是两个基础操作：
statck(堆叠) 该操作会“旋转”成将列中的数据透视到行
unstack(拆叠) 该操作会将行中的数据透视到列

In [39]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                    name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [40]:
# stack方法会将列透视到行，产生一个新的series
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [41]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [42]:
# 默认情况下，最内层是已拆队的（与stack方法一样）。你可以传入一个层级序号或名称来拆分一个不同的层级
print(result.unstack(0))
print(result.unstack('state'))

state   Ohio  Colorado
number                
one        0         3
two        1         4
three      2         5
state   Ohio  Colorado
number                
one        0         3
two        1         4
three      2         5


In [46]:
# 如果层级中的所有值并未包含于每个子分组中时，拆分可能会引入缺失值
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
print(data2)
print(data2.unstack())

# 默认情况下，堆叠会过滤出缺失值，因此堆叠拆堆的操作是可逆的
print(data2.unstack().stack())
print(data2.unstack().stack(dropna=False))

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64
       a    b    c    d    e
one  0.0  1.0  2.0  3.0  NaN
two  NaN  NaN  4.0  5.0  6.0
one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64
one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64


In [50]:
# 当在DataFrame中拆堆时，被拆堆的层级会变成结果在最低的层级
df = pd.DataFrame({'left':result, 'right':result+5}, columns=pd.Index(['left', 'right'], name='side'))
print(df)
print(df.unstack('state'))

# 在调用stack方法时，我们可以知名需要堆叠的轴向名称
print(df.unstack('state').stack('side'))

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10
side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10
state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7


8.3.2 将“长”透视为“宽”
在数据库和CSV中存储多时间序列的方式就是所谓的长格式或堆叠格式。

In [66]:
data = pd.read_csv('examples/macrodata.csv')
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [67]:
peroid = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')  # peroidindex 将year和quarter等列进行联合并生成一种时间间隔类型
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columns)  # reindex的作用是按照原有的列进行重新生成一个新的df
print(data.head())
data.index = peroid.to_timestamp('D', 'end')
print(data.head())
ldata = data.stack().reset_index().rename(columns={0: 'value'})  # reset_index生成新的index，原来的一列index现在变成了columns之一

ldata.head(10)  
# 这种数据即所谓的多时间序列的长格式，或称为具有两个或更多个键的其他观测数据。
# 数据通常以这种方式存储在关系型数据库中，比如MySQL,因为固定模式（列名称和数据类型）允许item列中不同值的而数量随着数据被添加到表中而改变

item   realgdp  infl  unemp
0     2710.349  0.00    5.8
1     2778.801  2.34    5.1
2     2775.488  2.74    5.3
3     2785.204  0.27    5.6
4     2847.699  2.31    5.2
item         realgdp  infl  unemp
date                             
1959-03-31  2710.349  0.00    5.8
1959-06-30  2778.801  2.34    5.1
1959-09-30  2775.488  2.74    5.3
1959-12-31  2785.204  0.27    5.6
1960-03-31  2847.699  2.31    5.2


,date,item,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,infl,0.000
2,1959-03-31,unemp,5.800
3,1959-06-30,realgdp,2778.801
4,1959-06-30,infl,2.340
5,1959-06-30,unemp,5.100
6,1959-09-30,realgdp,2775.488
7,1959-09-30,infl,2.740
8,1959-09-30,unemp,5.300
9,1959-12-31,realgdp,2785.204


In [68]:
# DataFrame中的pivot方法获取一个按date列时间戳索引的且每个不同的item独立一列的DataFrame
pivoted = ldata.pivot('date', 'item', 'value') # 传递的前两个值分别用作行和列索引，然后是可选的数值列以填充
pivoted.head(10)

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2
1960-06-30,0.14,2834.390,5.2
1960-09-30,2.70,2839.022,5.6
1960-12-31,1.21,2802.616,6.3
1961-03-31,-0.40,2819.264,6.8


In [69]:
# 假设存在两个数值列，相同时进行重塑
ldata['value2'] = np.random.randn(len(ldata))
ldata.head(10)

,date,item,value,value2
0,1959-03-31,realgdp,2710.349,0.800134
1,1959-03-31,infl,0.000,1.624857
2,1959-03-31,unemp,5.800,0.844235
3,1959-06-30,realgdp,2778.801,-1.521987
4,1959-06-30,infl,2.340,-0.111229
5,1959-06-30,unemp,5.100,0.044945
6,1959-09-30,realgdp,2775.488,-0.105066
7,1959-09-30,infl,2.740,0.298352
8,1959-09-30,unemp,5.300,-0.829452
9,1959-12-31,realgdp,2785.204,1.060995


In [70]:
# 如果省略最后一个参数，会得到一个含有多层列的DataFrame
pivoted = ldata.pivot('date', 'item')
print(pivoted[:5])
print(pivoted['value'][:5])

           value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8  1.624857  0.800134  0.844235
1959-06-30  2.34  2778.801   5.1 -0.111229 -1.521987  0.044945
1959-09-30  2.74  2775.488   5.3  0.298352 -0.105066 -0.829452
1959-12-31  0.27  2785.204   5.6  0.228899  1.060995 -0.283576
1960-03-31  2.31  2847.699   5.2  1.065592 -1.549412 -0.053326
item        infl   realgdp  unemp
date                             
1959-03-31  0.00  2710.349    5.8
1959-06-30  2.34  2778.801    5.1
1959-09-30  2.74  2775.488    5.3
1959-12-31  0.27  2785.204    5.6
1960-03-31  2.31  2847.699    5.2


In [72]:
# pivot方法等价于使用set_index创建分层索引，然后调用unstack
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8  1.624857  0.800134  0.844235
1959-06-30  2.34  2778.801   5.1 -0.111229 -1.521987  0.044945
1959-09-30  2.74  2775.488   5.3  0.298352 -0.105066 -0.829452
1959-12-31  0.27  2785.204   5.6  0.228899  1.060995 -0.283576
1960-03-31  2.31  2847.699   5.2  1.065592 -1.549412 -0.053326
1960-06-30  0.14  2834.390   5.2  0.091920 -1.412616  0.627906
1960-09-30  2.70  2839.022   5.6  0.802414 -0.271027  0.322839

8.3.3 将“宽”透视为“长”
在DataFrame中，与pivot方法的相反的是melt。与将一列变换为新的DataFrame中的多列不同，它将多列合并成一列，产生一个新的DataFrame，其长度比输入更长。

In [73]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [74]:
melted = pd.melt(df, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [76]:
# 使用Pivot方法，我们可以将数据重塑回原先的布局
reshaped = melted.pivot('key', 'variable', 'value')
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [77]:
# 可以指定列的子集作为值列
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [78]:
# 可以使用无需任何分组的指标
pd.melt(df, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [79]:
pd.melt(df, value_vars=['key', 'A', 'B', 'C'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
9,C,7
